In [1]:
import pandas as pd


In [3]:
df= pd.read_csv("/root/code/mrdaraujo/business_case_869/map_base.csv")
df

,id,date,store_nbr,family,sales,city,month,year,Lat,Lon
0,0,2013-01-01,1,AUTOMOTIVE,0.000,Quito,1,2013,-0.178352,-78.45879
1,1,2013-01-01,1,BABY CARE,0.000,Quito,1,2013,-0.178352,-78.45879
2,2,2013-01-01,1,BEAUTY,0.000,Quito,1,2013,-0.178352,-78.45879
3,3,2013-01-01,1,BEVERAGES,0.000,Quito,1,2013,-0.178352,-78.45879
4,4,2013-01-01,1,BOOKS,0.000,Quito,1,2013,-0.178352,-78.45879
...,...,...,...,...,...,...,...,...,...,...
2596369,2596237,2016-12-31,54,POULTRY,173.580,El Carmen,12,2016,-0.215621,-79.45972
2596370,2596238,2016-12-31,54,PREPARED FOODS,52.000,El Carmen,12,2016,-0.215621,-79.45972
2596371,2596239,2016-12-31,54,PRODUCE,1434.682,El Carmen,12,2016,-0.215621,-79.45972
2596372,2596240,2016-12-31,54,SCHOOL AND OFFICE SUPPLIES,0.000,El Carmen,12,2016,-0.215621,-79.45972


In [4]:
df.to_parquet("map_base.parquet")

In [5]:
pd.read_parquet("map_base.parquet")

,id,date,store_nbr,family,sales,city,month,year,Lat,Lon
0,0,2013-01-01,1,AUTOMOTIVE,0.000,Quito,1,2013,-0.178352,-78.45879
1,1,2013-01-01,1,BABY CARE,0.000,Quito,1,2013,-0.178352,-78.45879
2,2,2013-01-01,1,BEAUTY,0.000,Quito,1,2013,-0.178352,-78.45879
3,3,2013-01-01,1,BEVERAGES,0.000,Quito,1,2013,-0.178352,-78.45879
4,4,2013-01-01,1,BOOKS,0.000,Quito,1,2013,-0.178352,-78.45879
...,...,...,...,...,...,...,...,...,...,...
2596369,2596237,2016-12-31,54,POULTRY,173.580,El Carmen,12,2016,-0.215621,-79.45972
2596370,2596238,2016-12-31,54,PREPARED FOODS,52.000,El Carmen,12,2016,-0.215621,-79.45972
2596371,2596239,2016-12-31,54,PRODUCE,1434.682,El Carmen,12,2016,-0.215621,-79.45972
2596372,2596240,2016-12-31,54,SCHOOL AND OFFICE SUPPLIES,0.000,El Carmen,12,2016,-0.215621,-79.45972


In [8]:
df_2 = pd.read_csv("/root/code/mrdaraujo/business_case_869/data_train_merge_stores.csv")

In [9]:
df_2

,date,store_nbr,family,sales,city,month,year
0,2013-01-01,1,AUTOMOTIVE,0.000,Quito,1,2013
1,2013-01-01,1,BABY CARE,0.000,Quito,1,2013
2,2013-01-01,1,BEAUTY,0.000,Quito,1,2013
3,2013-01-01,1,BEVERAGES,0.000,Quito,1,2013
4,2013-01-01,1,BOOKS,0.000,Quito,1,2013
...,...,...,...,...,...,...,...
2596369,2016-12-31,9,POULTRY,687.853,Quito,12,2016
2596370,2016-12-31,9,PREPARED FOODS,100.405,Quito,12,2016
2596371,2016-12-31,9,PRODUCE,3091.356,Quito,12,2016
2596372,2016-12-31,9,SCHOOL AND OFFICE SUPPLIES,2.000,Quito,12,2016


In [10]:
df_2.to_parquet("data_train_merge_stores.parquet")

In [11]:
pd.read_parquet("data_train_merge_stores.parquet")

,date,store_nbr,family,sales,city,month,year
0,2013-01-01,1,AUTOMOTIVE,0.000,Quito,1,2013
1,2013-01-01,1,BABY CARE,0.000,Quito,1,2013
2,2013-01-01,1,BEAUTY,0.000,Quito,1,2013
3,2013-01-01,1,BEVERAGES,0.000,Quito,1,2013
4,2013-01-01,1,BOOKS,0.000,Quito,1,2013
...,...,...,...,...,...,...,...
2596369,2016-12-31,9,POULTRY,687.853,Quito,12,2016
2596370,2016-12-31,9,PREPARED FOODS,100.405,Quito,12,2016
2596371,2016-12-31,9,PRODUCE,3091.356,Quito,12,2016
2596372,2016-12-31,9,SCHOOL AND OFFICE SUPPLIES,2.000,Quito,12,2016
